In [1]:
import Pkg;
Pkg.activate(@__DIR__);
Pkg.instantiate()

  Activating environment at `~/Git/tinympc-julia/state_constraints/Project.toml`


In [1]:
using RobotZoo:Quadrotor
using RobotDynamics
using ForwardDiff
using TrajOptPlots
using BlockDiagonals
using LinearAlgebra
using StaticArrays
using SparseArrays
using OSQP

using Plots

using MeshCat
using ColorTypes
using GeometryBasics: HyperRectangle, Cylinder, Vec, Point, Mesh
using CoordinateTransformations
using Rotations

include("../quaternion-stuff.jl")

E (generic function with 1 method)

In [6]:
vis = Visualizer()
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /home/sam/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


false

In [7]:
# Quadrotor parameters
m = 0.035
ℓ = 0.046/sqrt(2)
J = [16.6e-6 0.83e-6 0.72e-6;
    0.83e-6 16.6e-6 1.8e-6;
    0.72e-6 1.8e-6 29.3e-6];
g = 9.81
# thrustToTorque = 0.005964552
thrustToTorque = 0.0008
scale = 65535
kt = 2.245365e-6*scale # u is PWM in range [0...1]
# kt=1
km = kt*thrustToTorque #4.4733e-8

h = 1/500

Nx = 13     # number of states (quaternion)
Nx̃ = 12     # number of states (linearized)
Nu = 4      # number of controls
nx = Nx̃
nu = Nu

#Goal state
@show uhover = (m*g/kt/4)*ones(4) # m = 30g and max thrust = 60g
# @show uhover = (m*g/4)*ones(4) # m = 30g and max thrust = 60g
rg = [0.0; 0; 0.0]
qg = [1.0; 0; 0; 0]
vg = zeros(3)
ωg = zeros(3)
xg = [rg; qg; vg; ωg];

uhover = (((m * g) / kt) / 4) * ones(4) = [0.5833333520642209, 0.5833333520642209, 0.5833333520642209, 0.5833333520642209]


In [8]:
function quad_dynamics(x,u)
    r = x[1:3]
    q = x[4:7]/norm(x[4:7]) #normalize q just to be careful
    v = x[8:10]
    ω = x[11:13]
    Q = qtoQ(q)
    
    ṙ = v
    q̇ = 0.5*L(q)*H*ω
    
    b = 1/m

    v̇ = [0; 0; -g] + (1/m)*Q*[zeros(2,4); kt*ones(1,4)]*u 
  
    ω̇ = J\(-hat(ω)*J*ω + [-ℓ*kt -ℓ*kt ℓ*kt ℓ*kt; -ℓ*kt ℓ*kt ℓ*kt -ℓ*kt; -km km -km km]*u)
    
    return [ṙ; q̇; v̇; ω̇]
  end
  function quad_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = quad_dynamics(x, u)
    f2 = quad_dynamics(x + 0.5*h*f1, u)
    f3 = quad_dynamics(x + 0.5*h*f2, u)
    f4 = quad_dynamics(x + h*f3, u)
    xn = x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
    xn[4:7] = xn[4:7]/norm(xn[4:7]) #re-normalize quaternion
    return xn
  end

quad_dynamics_rk4 (generic function with 1 method)

In [4]:
#Linearize dynamics about hover
Adyn = ForwardDiff.jacobian(x->quad_dynamics_rk4(x,uhover),xg)
Bdyn = ForwardDiff.jacobian(u->quad_dynamics_rk4(xg,u),uhover);
Ãdyn = Array(E(qg)'*Adyn*E(qg))
B̃dyn = Array(E(qg)'*Bdyn);

In [9]:
# Objective function
max_dev_x = [0.01; 0.01; 0.01; 0.5; 0.5; 0.05;  0.5; 0.5; 0.5;  0.7; 0.7; 0.5]
max_dev_u = [0.5; 0.5; 0.5; 0.5]
Q = spdiagm(1 ./(max_dev_x.^2))
R = spdiagm(1 ./(max_dev_u.^2))
QN = Q

Tfinal = 4.0
N = Int(Tfinal/h)+1

# Initial and reference states
X̃ref = [zeros(nx) for i = 1:N]
# Xref = [[sin(2*t)/2; cos(t); 0.3*sin(t)+1; 1; zeros(9)] for t = range(-pi/2, 3*pi/2, length = N)];
Xref = [[0.01; t; 0.5; 1; zeros(9)] for t = range(0, 5, length = N)];

# Compute reference velocity from reference position
for i = 1:N-1
    Xref[i][8:10] = (Xref[i+1][1:3] - Xref[i][1:3])/h
end

for k = 1:N
    x = Xref[k]
    qx = x[4:7]
    ϕ = qtorp(L(qg)'*qx)   
    X̃ref[k] = [x[1:3]; ϕ; x[8:10]; x[11:13]]
end

x0 = X̃ref[1] + [.2; 0; 0; zeros(9)]

# Visualize reference
delete!(vis["XrefLine"])
XrefLine = [Point(x_[1], x_[2], x_[3]) for x_ in Xref]
setobject!(vis["XrefLine"], Object(PointCloud(XrefLine), 
        LineBasicMaterial(color=Colors.RGBA(0.,0.45,1.)), "Line"))

;

In [10]:
# Create obstacle in MeshCat
# Cylinder constraint
c = [0.0; 2.0]
r_vis = 0.1
r = 0.4
setobject!(vis["cylinder"], Cylinder(Point(c[1], c[2], 0.0), Point(c[1], c[2], 1.0), r_vis))
bounding_cylinder_material = MeshPhongMaterial(color=RGBA(1, 1, 1, 0.25))
setobject!(vis["bounding cylinder"], Cylinder(Point(c[1], c[2], -.05), Point(c[1], c[2], 1.05), r), bounding_cylinder_material)

MeshCat Visualizer with path /meshcat/bounding cylinder at http://127.0.0.1:8703

In [11]:
function TransformPlane(visObject, center, dir)
    # dir[1] = x, dir[2] = y (for three dimensions, take cross product of [1,0,0] and 3D dir vector to get axis of rotation)
    width = 2
    height = 1
    dir_inv = -[dir[2]; -dir[1]]/norm(dir)
    bl = [center[1:2] + dir_inv*width/2; 0] # bottom left corner
    settransform!(visObject, Translation(bl[1], bl[2], bl[3]) ∘ LinearMap(AngleAxis(-atan(dir[2], -dir[1]), 0, 0, 1)))
end

TransformPlane (generic function with 1 method)

In [12]:
speye(N) = spdiagm(ones(N))

# Prediction horizon
Nh = 400

skip_vis = 1

# Create constraint planes
thin_rect_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.25))
constraintPlanes = [setobject!(vis["linearized constraint " * string(k)], HyperRectangle(Vec(0.0,0.0,0.0), Vec(0.001,2.0,1.0)), thin_rect_material) for k in 1:skip_vis:Nh+1]

# Constraints
umin = -[1.0; 1.0; 1.0; 1.0]*.5
umax = [1.0; 1.0; 1.0; 1.0]*.5
xmin = -Inf .* ones(12)
xmax = Inf .* ones(12)

# Cast MPC problem to a QP: x = (x(0),x(1),...,x(Nh),u(0),...,u(Nh-1))
# - quadratic objective
P = blockdiag(kron(speye(Nh), Q), QN, kron(speye(Nh), R))
# - linear objective
q = vec([reshape(hcat([(-Q*X̃ref[k]) for k = 1:Nh]...), (Nh*nx, 1)); -QN*X̃ref[Nh]; zeros(Nh*nu)])
# - linear dynamics
Ax = kron(speye(Nh + 1), -speye(nx)) + kron(spdiagm(-1 => ones(Nh)), Ãdyn)
Bu = kron([spzeros(1, Nh); speye(Nh)], B̃dyn)
Aeq = [Ax Bu]
leq = [-x0; zeros(Nh * nx)]
ueq = leq
# - input and state constraints
# Linearize cylinder into inequality constraints
Aineq = speye((Nh + 1) * nx + Nh * nu)
lineq = [repeat(xmin, Nh + 1); repeat(umin, Nh)]
uineq = [repeat(xmax, Nh + 1); repeat(umax, Nh)]
for k = 1:skip_vis:Nh+1
    x = X̃ref[k][1:2]
    xc = c - x
    a = xc/norm(xc)

    ind = (k-1)*nx
    Aineq[ind+1:ind+2, ind+1:ind+2] = [a'; 0 0]

    q_c = c - r*a
    b = a'*q_c
    uineq[ind+1] = b
    TransformPlane(constraintPlanes[cld(k,skip_vis)], q_c, a)
end
# - OSQP constraints
A, l, u = [Aeq; Aineq], [leq; lineq], [ueq; uineq]

# Create an OSQP model
model = OSQP.Model()

# Setup workspace
OSQP.setup!(model; P=P, q=q, A=A, l=l, u=u, warm_start=true, verbose=false)

# Simulate in closed loop
nsim = N - Nh;

x_linearized = [Point(0.0, 0.0, 0.0) for xp in 1:(nsim-1)*Nh]

xhist = [zeros(nx+1) for _ in 1:nsim]
xhist[1] = [x0[1:3]; rptoq(x0[4:6]); x0[7:12]]
uhist = [zeros(nu) for _ in 1:nsim-1]
@time for k in 1:nsim-1
# @time for k in 1:5
    # Solve
    res = OSQP.solve!(model)

    # Check solver status
    if res.info.status != :Solved
        error("OSQP did not solve the problem!")
    end

    # Apply first control input to the plant
    Δu = res.x[(Nh+1)*nx+1:(Nh+1)*nx+nu]
    uhist[k] = Δu + uhover
    xhist[k+1] = quad_dynamics_rk4(xhist[k], uhist[k])

    # Update initial state
    x_ = [xhist[k+1][1:3]; qtorp(xhist[k+1][4:7]); xhist[k+1][8:13]]
    # l[1:nx], u[1:nx] = -x_, -x_
    leq[1:nx], ueq[1:nx] = -x_, -x_

    Δu_array = reshape(res.x[(Nh+1)*nx+1:end], (4, Nh))

    # Roll out with real dynamics
    x_real = [zeros(nx+1) for _ = 1:Nh+1]
    x_real[1] = xhist[k]
    for j = 1:Nh
        x_real[j+1] = quad_dynamics_rk4(x_real[j], Δu_array[:,j] + uhover)
    end
    
    # Linearize cylinder into inequality constraints
    Aineq = speye((Nh + 1) * nx + Nh * nu)
    lineq = [repeat(xmin, Nh + 1); repeat(umin, Nh)]
    uineq = [repeat(xmax, Nh + 1); repeat(umax, Nh)]
    for j = 1:skip_vis:Nh+1
        # x = X̃ref[k+j-1][1:2]
        x = x_real[j][1:2]
        xc = c - x
        a = xc/norm(xc)

        ind = (j-1)*nx
        Aineq[ind+1:ind+2, ind+1:ind+2] = [a'; 0 0]

        q_c = c - r*a
        b = a'*q_c
        uineq[ind+1] = b
    
        TransformPlane(constraintPlanes[cld(j,skip_vis)], q_c, a)
    end
    # - OSQP constraints
    A, l, u = [Aeq; Aineq], [leq; lineq], [ueq; uineq]

    # Update reference trajectory (only q needs to be updated for this)
    q = vec([reshape(hcat([(-Q*X̃ref[j]) for j = k:k+Nh-1]...), (Nh*nx, 1)); -QN*X̃ref[k+Nh]; zeros(Nh*nu)])

    OSQP.update!(model; l=l, u=u, Ax=A.nzval, q=q)
    # OSQP.update!(model; l=l, u=u, q=q)
    
    # Visualize solution x
    delete!(vis["xLinearized"])
    # b = (k<=Nh) ? 1 : k-Nh
    # x_linearized = [Point(xp[1], xp[2], xp[3]+0.01) for xp in xhist[b:k]]
    x_linearized[k:k+Nh] = [Point(xp[1], xp[2], xp[3]+0.01) for xp in x_real]
    if k > 1
        x_lin_name_prev = "xlin " * string(k-1)
        setvisible!(vis[x_lin_name_prev], false)
    end
    x_lin_name = "xlin " * string(k)
    setobject!(vis[x_lin_name], Object(PointCloud(x_linearized[k:k+Nh]), 
        LineBasicMaterial(color=Colors.RGBA(1,0.6,0.0)), "Line"))

    xbar = RBState(quad_model, xhist[k+1])
    if quad_model.ned
        rq = position(xbar)
        vq = linear_velocity(xbar)
        rq = SA[rq[1],-rq[2],-rq[3]]
        vq = SA[vq[1],-vq[2],-vq[3]]
        xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
    end
    settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
end

178.087871 seconds (783.92 M allocations: 140.107 GiB, 5.36% gc time, 1.92% compilation time)


In [30]:
# Create MeshCat animation
anim = MeshCat.Animation()

for k = 1:nsim-1
    atframe(anim, k-1) do
        # Remove all horizon lines
        if k == 1
            for j = 1:nsim
                x_lin_name = "xlin " * string(j)
                setvisible!(vis[x_lin_name], false)
            end
        end

        # Update cylinder constraint visuals
        for j = 1:skip_vis:Nh+1
            if k+j-1 <= length(xhist)
                x = xhist[k+j-1][1:2]
            else
                x = xhist[end][1:2]
            end
            xc = c - x
            a = xc/norm(xc)
            q_c = c - r*a
            TransformPlane(constraintPlanes[cld(j,skip_vis)], q_c, a)
        end

        # Update visible horizon
        x_lin_name = "xlin " * string(k)
        setvisible!(vis[x_lin_name], true)
        if k > 1
            x_lin_name_prev = "xlin " * string(k-1)
            setvisible!(vis[x_lin_name_prev], false)
        end

        # Update quadrotor transform
        xbar = RBState(quad_model, xhist[k])
        if quad_model.ned
            rq = position(xbar)
            vq = linear_velocity(xbar)
            rq = SA[rq[1],-rq[2],-rq[3]]
            vq = SA[vq[1],-vq[2],-vq[3]]
            xbar = RBState(rq, RotX(pi)*orientation(xbar), vq, angular_velocity(xbar)) 
        end
        settransform!(vis["robot"], Translation(position(xbar)) ∘ LinearMap(UnitQuaternion(orientation(xbar))))
    end
end

setanimation!(vis, anim)

In [13]:
for i in length(constraintPlanes)
    delete!(constraintPlanes[i])
end

In [534]:
delete!(vis["xHistline"])
xHistline = [Point(x_[1], x_[2], x_[3]) for x_ in xhist]
setobject!(vis["xHistline"], Object(PointCloud(xHistline), 
LineBasicMaterial(color=Colors.RGBA(0.75,0.5,0.8)), "Line"))

X1 = [SVector{13}(x_) for x_ in xhist];
# X1 = [SVector{13}([x_[1:3]; rptoq(x_[4:6]); x_[7:12]]) for x_ in xhist];
visualize!(vis, quad_model, nsim*h, X1)

In [32]:
MeshCat.convert_frames_to_video(
    "/home/sam/Git/tinympc-julia/state_constraints/state_constraints_osqp.tar")

ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 8.1.0 (GCC)
  configuration: --enable-cross-compile --cross-prefix=/opt/x86_64-linux-gnu/bin/x86_64-linux-gnu- --arch=x86_64 --target-os=linux --cc=cc --cxx=c++ --dep-cc=cc --ar=ar --nm=nm --sysinclude=/workspace/destdir/include --pkg-config=/usr/bin/pkg-config --pkg-config-flags=--static --prefix=/workspace/destdir --sysroot=/opt/x86_64-linux-gnu/x86_64-linux-gnu/sys-root --extra-libs=-lpthread --enable-gpl --enable-version3 --enable-nonfree --disable-static --enable-shared --enable-pic --disable-debug --disable-doc --enable-avresample --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libvorbis --enable-libx264 --enable-libx265 --enable-libvpx --enable-encoders --enable-decoders --enable-muxers --enable-demuxers --enable-parsers --enable-openssl --disable-schannel --extra-cflags=-I/workspace/destdir/include --extra-ldflags=-L/workspac

Input #0, image2, from '%07d.png':
  Duration: 00:00:12.80, start: 0.000000, bitrate: N/A
  Stream #0:0: Video: png, rgba(pc), 1536x841, 25 fps, 25 tbr, 25 tbn, 25 tbc
Stream mapping:
  Stream #0:0 -> #0:0 (png (native) -> h264 (libx264))
Press [q] to stop, [?] for help


[libx264 @ 0x1c0fe80] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x1c0fe80] profile High 4:4:4 Predictive, level 4.2, 4:4:4, 8-bit
[libx264 @ 0x1c0fe80] 264 - core 163 - H.264/MPEG-4 AVC codec - Copyleft 2003-2021 - http://www.videolan.org/x264.html - options: cabac=1 ref=5 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=8 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=2 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=24 lookahead_threads=4 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=3 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=50 rc=crf mbtree=1 crf=18.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to '/home/sam/Git/tinympc-julia/state_constraints/output.mp4':
  Metadata:
    encoder         : Lavf58.76.100
  Stream #0:0: Video: h264 (avc1 

frame=  321 fps= 67 q=-1.0 Lsize=     879kB time=00:00:05.30 bitrate=1357.8kbits/s dup=1 drop=0 speed= 1.1x    
video:874kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.526218%
[libx264 @ 0x1c0fe80] frame I:2     Avg QP:14.00  size:225488
[libx264 @ 0x1c0fe80] frame P:81    Avg QP:19.23  size:  2490
[libx264 @ 0x1c0fe80] frame B:238   Avg QP:27.23  size:  1015
[libx264 @ 0x1c0fe80] consecutive B-frames:  0.9%  0.6%  0.0% 98.4%
[libx264 @ 0x1c0fe80] mb I  I16..4: 24.3% 18.8% 56.9%
[libx264 @ 0x1c0fe80] mb P  I16..4:  0.2%  0.2%  0.2%  P16..4:  2.2%  0.8%  0.6%  0.0%  0.0%    skip:95.8%
[libx264 @ 0x1c0fe80] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8:  4.1%  0.4%  0.1%  direct: 0.2%  skip:95.1%  L0:54.6% L1:41.7% BI: 3.7%
[libx264 @ 0x1c0fe80] 8x8 transform intra:25.2% inter:52.0%
[libx264 @ 0x1c0fe80] direct mvs  spatial:95.0% temporal:5.0%
[libx264 @ 0x1c0fe80] coded y,u,v intra: 39.2% 35.2% 32.8% inter: 0.3% 0.6% 0.5%
[libx264 @ 0x1c0fe80] i16 v,h,dc,p: 

"/home/sam/Git/tinympc-julia/state_constraints/output.mp4"